# Clean products

## Initialize Cleaning

### Import Packages and Raw_data

In [370]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

# Global variables
df_name = "products" 
data_path = '../../data/raw_data/' 
cl_data_path = '../../data/clean_data/' 

init_df = pd.read_csv(f"{data_path}{df_name}.csv")
o_lines_cleaned = pd.read_csv(f"{cl_data_path}clean_orderlines.csv")
brands_cleaned = pd.read_csv(f"{cl_data_path}clean_brands.csv")

o_lines_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293982 entries, 0 to 293981
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   o_lines_id           293982 non-null  int64  
 1   order_id             293982 non-null  int64  
 2   o_lines_product_qty  293982 non-null  int64  
 3   sku                  293982 non-null  object 
 4   o_lines_unit_price   293982 non-null  float64
 5   o_lines_date         293982 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 13.5+ MB


### Logging

In [371]:
from functools import wraps
import datetime as dt

def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

## Pipeline - Functions

### Start Pipeline

In [372]:
@log_step
def start_pipeline(df):
    return df.copy()

### Rename Columns

In [373]:
init_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


In [374]:
def rename_columns(df):
    return (df
        .rename(columns={
            "name": "product_name",
            "desc":  "product_description",      
            "price":"product_price_dirty",        
            "in_stock":"product_in_stock"
            }
        )
    )

### Remove duplicates

In [375]:
@log_step
def remove_duplicates(df):
    return (
            df
            .drop_duplicates()       # removed dupliceted rows
            .drop_duplicates("sku")  # one duplictaed sku. 2. Appearence has no price --> dropped!
    )

### Adjust price

In [386]:
@log_step
def adjust_price(df):
    # make sure eveything is a string
    df["product_price_dirty"] = df["product_price_dirty"].astype("string")
    df["product_price_dirty"] = df["product_price_dirty"].str.replace(".*\.\d{3}$", "???", regex = True)
    df["product_price_dirty"] = df["product_price_dirty"].str.replace("^0*\.?(0*)?$", "???", regex = True)

    # get maximum price from orderlines
    max_o_lines_df = (
        o_lines_cleaned[["sku","o_lines_unit_price"]]
            .groupby("sku")
            .agg("max")
            .reset_index()
    )
    df = df.merge(max_o_lines_df, how="right", on="sku")
    df = df.rename(columns={"o_lines_unit_price":"max_price_o_lines"})
    # assign cleaned price
    df = df.assign(product_price = lambda x: x["max_price_o_lines"] if x["product_price_dirty"].str == "\?\?\?" else x["product_price_dirty"])
    
    return df
    
    

### Missing Values

In [377]:
@log_step
def missing_values(df):
    return df.fillna({"product_price_dirty":"000.000"})

### Adjust data types

In [378]:
@log_step
def adjust_data_types(df):
   df["sku"] = df["sku"].astype("string")
   df["product_name"] = df["product_name"].astype("string")
   df["product_description"] = df["product_description"].astype("string")
   df["product_brand"] = df["product_brand"].astype("category")
   return df

### Drop Columns

In [379]:
@log_step
def drop_columns(df, cols):
    
    return df.drop(columns=cols)


### Add columns

In [380]:
@log_step
def add_columns(df):
    # extract brand_short from sku
    df = df.assign(brand_short = df["sku"].str.extract("(^\w{3})"))
    
    # merge brands on "brands_short"

    df = df.merge(brands_cleaned, how="left", on="brand_short")
    df = df.rename(columns={"brand_long":"product_brand"})
    return df

### CSV

In [381]:
@log_step
def create_csv(df):
    df.to_csv(f"../../data/clean_data/clean_{df_name}.csv", index = False)
    return df

### Report

In [382]:
@log_step
def create_report(df):
    ProfileReport(df, title=f"{df_name.title()} - Report", dark_mode=True, minimal = True).to_file(f"../../data/profile_reports/{df_name}_report.html")
    return df

## Run Cleaning

In [387]:
products = (
init_df
    .pipe(start_pipeline)       # copy df
    .pipe(rename_columns)       # naming conventions
    .pipe(missing_values)       # price --> replaced with: "000.000"
    .pipe(remove_duplicates)    # duplicated rows + duplicated sku, removed first one, without a price
    .pipe(adjust_price)         # removed second dot: 1.000.99 -> 1000.99  + to_numeric 
    .pipe(add_columns)          # extract brand short from sku --> merge brand_long from brands
    .pipe(drop_columns, cols=["promo_price", "type","brand_short"])
    .pipe(adjust_data_types)    # product_id -> STRING, order_id --> STRING, product_date --> DATETIME
    .pipe(create_csv)           # save clean data as .csv 
    .pipe(create_report)        # save report as .html 

)


start_pipeline:
 shape=(19326, 7) took 0:00:00.000949s

missing_values:
 shape=(19326, 7) took 0:00:00.003266s

remove_duplicates:
 shape=(10579, 7) took 0:00:00.022131s

adjust_price:
 shape=(7951, 9) took 0:00:00.101404s

add_columns:
 shape=(7951, 11) took 0:00:00.029747s

drop_columns:
 shape=(7951, 8) took 0:00:00.001276s

adjust_data_types:
 shape=(7951, 8) took 0:00:00.005537s

create_csv:
 shape=(7951, 8) took 0:00:00.062833s



Export report to file: 100%|██████████| 1/1 [00:00<00:00, 834.02it/s]

create_report:
 shape=(7951, 8) took 0:00:01.511027s



## Profile Report

### Types

#### Before

In [ ]:
init_df.dtypes

sku            object
name           object
desc           object
price          object
promo_price    object
in_stock        int64
type           object
dtype: object

#### After

In [ ]:
products.dtypes

sku                      string
product_name             string
product_description      string
product_price_dirty      string
product_in_stock        float64
max_price_o_lines       float64
product_price            string
product_brand          category
dtype: object

### Samples

#### Before

In [ ]:
init_df.sample(10)

,sku,name,desc,price,promo_price,in_stock,type
14284,APP1916,"Apple MacBook Pro 15 ""Core i7 Touch Bar 27GHz ...",New MacBook Pro 15-inch Touch Bar to 27GHz Cor...,4639,44.135.839,0,2158
6801,PAC1607,"Apple iMac 27 ""Core i5 3.3GHz Retina 5K | 8GB ...",Desktop computer iMac 27-inch 3.3GHz Core i5 5...,3349,29.099.895,0,"5,74E+15"
9631,PAC0963,"Apple iMac 27 ""Core i7 Retina 5K 4GHz | 32GB |...",IMac desktop computer 27 inch 5K Retina 4GHz i...,3409,28.569.903,0,"5,74E+15"
14780,PAC1711,Pack QNAP TS-451A NAS 8GB RAM l l 8TB (4x2TB) ...,Nas TS-451A with 8GB RAM and 8TB (4x2TB) Seaga...,10.008.289,8.393.685,0,12175397
4166,APP1389,"Apple iMac 27 ""Core i5 3.3GHz Retina 5K | 8GB ...",IMac desktop computer 27 inch 8GB RAM 512GB Re...,3169,30.175.839,0,"5,74E+15"
3946,APP1385,"Apple iMac 27 ""Core i5 3.3GHz Retina 5K | 8GB ...",IMac desktop computer 27 inch 5K Retina 8GB RA...,3049,2.903.585,0,"5,74E+15"
13300,UAG0051,Urban Armor Gear iPhone Case Plasma 7 Red / Black,Cover with military certificate falls and anti...,29.95,269.903,0,11865403
5557,PAC1054,"Apple iMac 27 ""Core i5 3.3GHz Retina 5K | 32GB...",IMac desktop computer 27 inch 5K Retina i5 3.3...,3649,30.809.903,0,"5,74E+15"
13363,APP1630,Apple iPhone 7 Plus 128GB Black,New Apple iPhone 7 Plus Free Black 128GB,889,872.001,1,85651716
12664,LIF0103-A,Open - LifeProof Fre Waterproof Case iPhone 6 ...,waterproof case and extreme conditions for iPh...,89.99,649.903,0,1298


#### After

In [ ]:

    products.sample(10).reset_index()
    

,index,sku,product_name,product_description,product_price_dirty,product_in_stock,max_price_o_lines,product_price,product_brand
0,2471,DLL0047,Dell E-series E2417H 24 FHD IPS DP VGA,Monitor 24 inch FHD with high vision support w...,196.6,0.0,176.99,196.6,Dell
1,7435,TUC0305-A,Open - Tucano Nido Hard-Shell Case MacBook Pro...,reconditioned protective housing with slip rub...,34.9,0.0,29.99,34.9,Tucano
2,7176,TCH0012,Tech21 Evo Check Case iPhone X Rosa,X iPhone case studied to overcome multiple dro...,39.99,1.0,34.99,39.99,Tech21
3,6890,SPE0179-A,Open - Speck iPhone Case Presidio 7 Plus Black...,cover slip with slim design and touch buttons ...,29.95,0.0,18.91,29.95,Spek SeeThru
4,6267,QNA0230,QNAP QM2-2P10G1T PCIe expansion card dual 10Gb...,PCIe expansion card to connect two M.2 SSD 10G...,258.69,0.0,257.99,258.69,QNAP
5,3999,MAK0020-A,Open - Maclocks Security Bundle Case iPad Air ...,Antirobo housing with support for iPad Air,000.000,0.0,59.85,000.000,Maclocks
6,2756,GRT0386-A,(Open) Griffin Survivor Case Tactical Apple Wa...,Anti-shock Case with Screen Protector for Appl...,34.99,0.0,12.66,34.99,Griffin
7,1954,BEL0264-A,Open - PowerHouse Base Load Belkin Apple Watch...,Reconditioned support charger and connection L...,99.99,0.0,62.25,99.99,Belkin
8,6328,REP0243,Microphone Repair iPhone 6,Repair service including parts and labor for i...,000.000,0.0,69.90,000.000,Repair
9,2701,GLY0014,Glyph StudioRAID System 7200rpm HDD 12TB RAID ...,12TB RAID system with free Thunderbolt 2 and 2...,1057.63,0.0,1007.80,1057.63,Glyph Atom


### Info - Types Range, Non Null Count, Dtype

#### Before

In [ ]:
init_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


#### After

In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7951 entries, 0 to 7950
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   sku                  7951 non-null   string  
 1   product_name         7850 non-null   string  
 2   product_description  7844 non-null   string  
 3   product_price_dirty  7850 non-null   string  
 4   product_in_stock     7850 non-null   float64 
 5   max_price_o_lines    7951 non-null   float64 
 6   product_price        7850 non-null   string  
 7   product_brand        7873 non-null   category
dtypes: category(1), float64(2), string(5)
memory usage: 518.0 KB
